In [187]:
from langgraph.graph import StateGraph,END,START
from typing_extensions import TypedDict

In [188]:
import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("groq_api_key")



In [ ]:
# Connecting LLM with Chatmodel
from langchain_groq.chat_models import ChatGroq
from langchain_core.messages import HumanMessage

llm =  ChatGroq(
    model_name = "gemma2-9b-it",
    groq_api_key= ""
)

In [190]:
response = llm.invoke(
    [HumanMessage(content ="Hi")]
)

In [191]:
# print(response)

In [192]:
class State(TypedDict):
    application :str
    experience_level :str
    skill_match : str
    response :str

In [193]:
workflow = StateGraph(State)

In [194]:
from langchain_core.prompts import ChatPromptTemplate

In [195]:
# Node 1 :'categorize_experience' to categorize the candidate exp.
def categorize_experience(state: State)->State:
    print("\nCategorizing the experience level of candidate:")
    prompt = ChatPromptTemplate.from_template(
        "Based on the following job application , categorize the candidate as 'Entry-level','Mid-level','Senior-level'"
        "Application:{application}"
        )
    chain = prompt | llm
    experience_level = chain.invoke({"application":state["application"]}).content
    print(f"Experience Level : {experience_level}")
    return {"experience_level":experience_level}


In [196]:
# Node 2 : 'assess_skillset' to 'match' or 'No match'
def asses_skillset(state : State)->State:
    print("\nAssessing the skillset of the candidate:")
    prompt = ChatPromptTemplate.from_template(
        "Based on the job application for a Python Developer, assess the the candidate's skillset"
        "Respond with either 'Match' or 'No-Match'"
        "Application : {application}"
    )
    chain = prompt | llm
    skill_match = chain.invoke({"application":state["application"]}).content
    print(f"Skill Match : {skill_match}")
    return {"skill_match": skill_match}

In [197]:
# Node 3 :Schedule_hr_interview on skill set Match.
def schedule_hr_interview(state: State)->State:
    print("\n Schedule the  interview")
    return {"response": "Candidate has been shortlisted for an HR interview."}

In [198]:
# Node 4 : Escalation case if exp of candidate is hight and not meeting the skills.
def escalate_to_recruiter(state : State)->State:
    print("\n Escalating to recruiter.")
    return {"response": "Candidate has senior_level experience but does't match the job skills."}

In [199]:
# Node 5 : Rejection case when skill does't meet the JD.
def reject_application(state:State)->State:
    print("Sending rejecting email.")
    return{"response":"Candidate does't meet the JD and has been rejected."}

In [200]:
workflow.add_node("categorize_experience",categorize_experience)

In [201]:
# Add remaining Nodes to the Graph
workflow.add_node("asses_skillset",asses_skillset)
workflow.add_node("schedule_hr_interview",schedule_hr_interview)
workflow.add_node("escalate_to_recruiter",escalate_to_recruiter)
workflow.add_node("reject_application",reject_application)

In [202]:
# def route_app(state:State)->str:
#     if(state["skill_match"] != "Match"):        
#         return "reject_application"
#     elif(state["experience_level"]=="senior-level"):
#         return "escalate_to_recruiter"
#     else:
#         return "schedule_hr_interview"

def route_app(state :State)->State:
    if(state["experience_level"]=="senior-level"):
        return "escalate_to_recruiter"
    elif(state["skill_match"]=="No-Match"):
        return "reject_application"
    else:
        return "schedule_hr_interview"

In [203]:
workflow.add_edge("categorize_experience","asses_skillset")
workflow.add_conditional_edges("asses_skillset",route_app)

In [204]:
workflow.add_edge(START,"categorize_experience")
workflow.add_edge("asses_skillset",END)
workflow.add_edge("escalate_to_recruiter",END)
workflow.add_edge("reject_application",END)
workflow.add_edge("schedule_hr_interview",END)

In [205]:
app = workflow.compile()

In [209]:

# from IPython.display import Image,display
# display(Image(app.get_graph().draw_mermaid_png()))

In [207]:
def run_candidate_screening(application:str):
    results = app.invoke({"application":application})
    return {
        "experience_level":results["experience_level"],
        "skill_match":results["skill_match"],
        "response":results["response"]
    }
                         

In [208]:
application_text = "I have 5 years of experience in application developement on Python."
results = run_candidate_screening(application_text)
print("\n\nComputed Results:")
print(f"Application:{application_text}")
print(f"Experience Level :{results['experience_level']}")
print(f"Skill Match :{results['skill_match']}")
print(f"Response :{response.content}")


Categorizing the experience level of candidate:
Experience Level : Based on the information provided, the candidate would be categorized as **Mid-level**. 

Here's why:

* **5 years of experience** is generally considered beyond the typical entry-level range. 
*  While not a significant number of years to be considered senior, it demonstrates a solid foundation and likely some project ownership or leadership experience. 


Let me know if you have any other application details you'd like me to consider for a more accurate assessment! 


Assessing the skillset of the candidate:
Skill Match : Match 


 Schedule the  interview


Computed Results:
Application:I have 5 years of experience in application developement on Python.
Experience Level :Based on the information provided, the candidate would be categorized as **Mid-level**. 

Here's why:

* **5 years of experience** is generally considered beyond the typical entry-level range. 
*  While not a significant number of years to be conside